In [1]:
!pip install google-cloud-core
!pip install google-cloud-bigtable

  Created wheel for grpc-google-iam-v1: filename=grpc_google_iam_v1-0.12.3-py3-none-any.whl size=18503 sha256=eba310724c5ee42dadbeb9e7d45cc52000081fe7d8bf1c681ab87d0bb355334d
  Stored in directory: c:\users\jeiso\appdata\local\pip\cache\wheels\8f\b9\13\fce3d62261f63c01b28281fe6a9d704a7af65d96ff2c88552e
Successfully built grpc-google-iam-v1


In [3]:
import time
from datetime import datetime as dt

from google.cloud import bigtable
from google.cloud.bigtable import column_family

In [4]:
INSTANCE_ID = 'bigtable-1'
TABLE_ID = 'orders_{}'.format(time.time())

In [5]:
TABLE_ID

'orders_1634492147.701365'

### Connecting to BigTable

In [6]:
client = bigtable.Client.from_service_account_json('../../unbosque-service-account.json', admin = True)

In [7]:
print('Connecting to database instance')
instance = client.instance(INSTANCE_ID)

Connecting to database instance


### Creating a table

In [ ]:
print('Creating the {} table'.format(TABLE_ID))
table = instance.table(TABLE_ID)

if not table.exists():
    table.create()
else:
    print("ERROR: Table {} already exists".format(TABLE_ID))

In [ ]:
print('Creating all family columns')

customer_cfId = 'customer'
customer_cf = table.column_family(customer_cfId)
customer_cf.create()

items_cfId = 'items'
items_cf = table.column_family(items_cfId)
items_cf.create()

delivery_cfId = 'delivery'
delivery_cf = table.column_family(delivery_cfId)
delivery_cf.create()

### Inserting data

In [ ]:
orders = [
    {
        'orderId': '2497',
        'customer': {
            'firstName': 'Fabian',
            'lastName': 'Penia',
            'customerId': '101'
        },
        'items': [
            {
                'productId': '3301',
                'quantity': '5',
                'unitPrice': '12000'
            },
            {
                'productId': '3302',
                'quantity': '1',
                'unitPrice': '18000'
            },
            {
                'productId': '3303',
                'quantity': '2',
                'unitPrice': '25000'
            }
        ],
        'delivery': {
            'addressLine1': 'Fake Street 123',
            'addressLine2': 'House 10',
            'city': 'Springfield',
            'country': 'US'
        }
    },
    {
        'orderId': '2498',
        'customer': {
            'firstName': 'Pepito',
            'lastName': 'Perez',
            'customerId': '102'
        },
        'items': [
            {
                'productId': '3301',
                'quantity': '1',
                'unitPrice': '12000'
            },
            {
                'productId': '3306',
                'quantity': '3',
                'unitPrice': '20000'
            }
        ],
        'delivery': {
            'addressLine1': '742 Evergreen Terrace',
            'city': 'Springfield',
            'country': 'US'
        }
    }
]

In [ ]:
dt = dt.utcnow()
rows = []

print('Writing orders to the table')
for order in orders:
    
    row_key = 'order#{}'.format(order['orderId']).encode()
    row = table.direct_row(row_key)
    
    row.set_cell(customer_cfId, 'firstName'.encode(), order['customer']['firstName'], timestamp = dt)
    row.set_cell(customer_cfId, 'lastName'.encode(), order['customer']['lastName'], timestamp = dt)
    row.set_cell(customer_cfId, 'customerId'.encode(), order['customer']['customerId'], timestamp = dt)
    
    for item in order['items']:
        row.set_cell(items_cfId, 'product#{}'.format(item['productId']).encode(), (item['quantity'] + ':' + item['unitPrice']), timestamp = dt)
    
    row.set_cell(delivery_cfId, 'addressLine1'.encode(), order['delivery']['addressLine1'], timestamp = dt)
    try:
        row.set_cell(delivery_cfId, 'addressLine2'.encode(), order['delivery']['addressLine2'], timestamp = dt)
    except KeyError:
        pass
    row.set_cell(delivery_cfId, 'city'.encode(), order['delivery']['city'], timestamp = dt)
    row.set_cell(delivery_cfId, 'country'.encode(), order['delivery']['country'], timestamp = dt)
    
    rows.append(row)

table.mutate_rows(rows)

### Reading data

In [ ]:
print('Getting a single order by row key')
key = 'order#2497'.encode()

row = table.read_row(key)
print_row(row)

In [ ]:
print('Getting a specific product item by row key')
key = 'order#2497'.encode()

col_filter = row_filters.ColumnQualifierRegexFilter('product#3302'.encode())

row = table.read_row(key, filter_ = col_filter)
print_row(row)

In [ ]:
print('Getting all orders')

rows = table.read_rows()
for row in rows:
    print_row(row)

In [ ]:
print('Getting a specific column for all orders')

col_filter = row_filters.ColumnQualifierRegexFilter('product#3302'.encode())

rows = table.read_rows(filter_ = col_filter)
for row in rows:
    print_row(row)

### Deleting a row

In [ ]:
print('Deleting a row by key')
key = 'order#2498'.encode()

row = table.direct_row(row_key)
row.delete()
row.commit()

In [ ]:
print('Getting all orders')

rows = table.read_rows()
for row in rows:
    print_row(row)

### Deleting table

In [ ]:
print('Deleting the {} table.'.format(TABLE_ID))
table.delete()

### Functions

In [ ]:
def print_row(row):
    print("Reading data for {}:".format(row.row_key.decode('utf-8')))
    for cf, cols in sorted(row.cells.items()):
        print("Column Family {}".format(cf))
        for col, cells in sorted(cols.items()):
            for cell in cells:
                labels = " [{}]".format(",".join(cell.labels)) \
                    if len(cell.labels) else ""
                print(
                    "\t{}: {} @{}{}".format(col.decode('utf-8'),
                                            cell.value.decode('utf-8'),
                                            cell.timestamp, labels))
    print("")

In [ ]:
firstName = row.cells[customer_cfId]['firstName'.encode()][0].value.decode('utf-8')
lastName = row.cells[customer_cfId]['lastName'.encode()][0].value.decode('utf-8')
customerId = row.cells[customer_cfId]['customerId'.encode()][0].value.decode('utf-8')

print(firstName + ' | ' + lastName + ' | ' + customerId)
print("\n")

for item in [k.decode('utf-8') for k in row.cells[items_cfId].keys()]:
    cell_content = row.cells[items_cfId][item.encode()][0].value.decode('utf-8')
    print(item + ' | ' + ' | '.join(cell_content.split(':')))
    
print("\n")

addressLine1 = row.cells[delivery_cfId]['addressLine1'.encode()][0].value.decode('utf-8')
addressLine2 = row.cells[delivery_cfId]['addressLine2'.encode()][0].value.decode('utf-8')
city = row.cells[delivery_cfId]['city'.encode()][0].value.decode('utf-8')
country = row.cells[delivery_cfId]['country'.encode()][0].value.decode('utf-8')

print(addressLine1 + ' | ' + addressLine2 + ' | ' + city + ' | ' + country)